
# Pre-processing Experiment

This notebook evaluates OWL-ViT detections after applying image pre-processing (histogram equalization, autocontrast, contrast enhancement, unsharp mask) to boost feature visibility.

In [ ]:
# Setup
import sys
from pathlib import Path
import torch

tools_loc = Path.cwd().resolve().parents[2]
sys.path.insert(0, str(tools_loc))
              
from tools.owlvit_utils   import OwlViTPipeline
from tools.preprocess_utils import PreprocessUtils

In [ ]:
# Experiment configuration
experiment_name      = 'preprocessing'
image_filename       = '<input_image_filename>'  # e.g., 'site_image.png'
prompts_file         = '<prompts_file>'          # e.g., 'prompts.txt'
resize_size          = (<width>, <height>)       # e.g., (1024, 1024)
thresholds           = [<threshold_1>, <threshold_2>, <threshold_3>, <threshold_4>]  # e.g., [0.001, 0.002, ...]

# Instantiate pipeline
pipeline = OwlViTPipeline(
    experiment_name=experiment_name,
    resize_size=resize_size
)

# Load original image & prompts
pipeline.load_image(image_filename, image_name=image_name)
pipeline.load_prompts(prompts_file)

In [ ]:
print(pipeline.prompts)

In [ ]:
# Choose which steps to apply
apply_equalize       = True
apply_autocontrast   = True
contrast_factor      = 1.8
apply_unsharp        = True

# Run the pipeline-wide preprocess
pre_img = PreprocessUtils.preprocess_pipeline(
    pipeline.image_full,
    equalize       = apply_equalize,
    autocontrast   = apply_autocontrast,
    contrast_factor= contrast_factor,
    unsharp        = apply_unsharp
)

# Visual check
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(pre_img)
ax.axis('off')
plt.show()

# Override pipeline.image_resized with preprocessed image
pipeline.image_resized = pre_img.resize(pipeline.resize_size)

In [ ]:
# Loop over thresholds
for thresh in thresholds:

    # the raw results dict:
    inputs = pipeline.processor(
        text    = pipeline.prompts,
        images  = pipeline.image_resized,
        return_tensors = "pt"
    )
    with torch.no_grad():
        outputs = pipeline.model(**inputs)
    results = pipeline.processor.post_process_object_detection(
        outputs,
        target_sizes=torch.tensor([[pipeline.image_full.height,
                                    pipeline.image_full.width]]),
        threshold=thresh
    )[0]
    
    png_path = pipeline.save_visualisation(
        results,
        threshold=thresh
    )
    print("Saved visualization to:", png_path)
    
    # Save the per-threshold metrics JSON
    json_path = pipeline.save_metrics(
        results,
        threshold=thresh,
        suffix='tiling'
    )
    print("Saved metrics to:", json_path)
    
    # save geojson
    geojson_path = pipeline.run_and_save_geojson(thresh)
    
    
